In [ ]:
import pygame
import sys
import os
import random
import datetime
import moviepy
from moviepy import VideoFileClip  # Importer moviepy pour lire la vidéo

# Hardcoded metadata values
app_name = "Joyeux Anniversaire!"

# Constantes
WHITE = (255, 255, 255)
SCREEN_WIDTH, SCREEN_HEIGHT = 400, 600
BIRD_X = 50
BIRD_Y = 300
GRAVITY = 0.25
FLAP_STRENGTH = -5
PIPE_SPEED_INIT = -4
PIPE_GAP = 150
PIPE_GAP_MIN = 180  # Valeur minimale de l'écart
PIPE_GAP_MAX = 200  # Valeur maximale de l'écart
SPEED_INCREASE_INTERVAL = 3000
SCORE_INTERVAL_FOR_SPEED_INCREASE = 5

FONT_NAME = pygame.font.match_font('arial')

def draw_text(surf, text, size, x, y, color=(255, 255, 255), font_name=FONT_NAME):
    font = pygame.font.Font(font_name, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    text_rect.midtop = (x, y)
    surf.blit(text_surface, text_rect)

def create_snowflakes(num_flakes):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT),
             random.randint(2, 5)] for _ in range(num_flakes)]

def update_and_draw_snowflakes(screen, snowflakes):
    for flake in snowflakes:
        flake[1] += flake[2]
        pygame.draw.circle(screen, WHITE, (flake[0], flake[1]), flake[2])
        if flake[1] > SCREEN_HEIGHT:
            flake[0] = random.randint(0, SCREEN_WIDTH)
            flake[1] = -5
            flake[2] = random.randint(2, 5)

def draw_button(surf, text, x, y, width, height):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    button_clicked = False

    if x + width > mouse[0] > x and y + height > mouse[1] > y:
        pygame.draw.rect(surf, (200, 200, 200), (x, y, width, height))
        if click[0] == 1:
            button_clicked = True
    else:
        pygame.draw.rect(surf, (170, 170, 170), (x, y, width, height))

    draw_text(surf, text, 20, x + width / 2, y + 10)
    return button_clicked

def start_screen(screen):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)
    intro_text = [
        "Bienvenue au jeu d'anniversaire!",
        "Aidez le personnage à atteindre",
        "le score de 10 pour une surprise spéciale!",
        "",
        "Utilisez la flèche du haut pour monter,",
        "et celle du bas pour descendre.",
        "Bonne chance!"
    ]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((102, 190, 209))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)

        draw_text(screen, "Joyeux Anniversaire!", 40, SCREEN_WIDTH / 2,
                  SCREEN_HEIGHT / 10, color=(255, 50, 0))
        y_offset = 150
        for line in intro_text:
            draw_text(screen, line, 20, SCREEN_WIDTH / 2, y_offset)
            y_offset += 30

        if draw_button(screen, "Commencer", 150, 450, 100, 50):
            break

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def game_over_screen(screen, final_score):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((102, 190, 209))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)  # Dessiner les flocons de neige

        draw_text(screen, "Game Over", 48, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 4)
        score_text = f"Score Final: {final_score}"
        draw_text(screen, score_text, 36, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50)

        if draw_button(screen, "Rejouer", 100, 350, 100, 50):
            return True  # Indique de recommencer le jeu
        if draw_button(screen, "Quitter", 200, 350, 100, 50):
            return False  # Indique de quitter le jeu

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def create_stars(num_stars):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT)] for _ in range(num_stars)]

def draw_stars(screen, stars):
    for star in stars:
        pygame.draw.circle(screen, (255, 255, 0), star, random.randint(1, 3))

def wait_for_key():
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                waiting = False

def play_video(screen, video_path):
    clip = VideoFileClip(video_path)
    clip = clip.resized(height=SCREEN_HEIGHT, width=SCREEN_WIDTH)
    clock = pygame.time.Clock()

    for frame in clip.iter_frames(fps=30, dtype="uint8"):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        frame_surface = pygame.surfarray.make_surface(frame.swapaxes(0, 1))
        screen.blit(frame_surface, (0, 0))
        pygame.display.update()
        clock.tick(30)
    clip.close()

# def draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img, plante_img, has_plant):
#     brique_height = brique_img.get_height()
#     brique_width = brique_img.get_width()
#     plante_height = plante_img.get_height()
#     plante_width = plante_img.get_width()

#     # Draw the bricks for the upper pipe (inverted)
#     y_top = pipe_y - pipe_gap - brique_height
#     while y_top > -brique_height:
#         screen.blit(pygame.transform.flip(brique_img, False, True), (pipe_x, y_top))
#         y_top -= brique_height

#     # Draw the bricks for the lower pipe
#     y_bottom = pipe_y
#     while y_bottom < SCREEN_HEIGHT:
#         screen.blit(brique_img, (pipe_x, y_bottom))
#         y_bottom += brique_height

#     # Draw the plant if the pipe has a plant
#     if has_plant:
#         # Draw the plant at the bottom of the upper pipe
#         # plant_position_y = pipe_y - pipe_gap - plante_height
#         # screen.blit(plante_img, (pipe_x + (brique_width - plante_width) // 2, plant_position_y))
#         screen.blit(plante_img, (pipe_x + (brique_width - plante_width) // 2, pipe_y - plante_height))

def draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img, plante_img, has_plant):
    brique_height = brique_img.get_height()
    brique_width = brique_img.get_width()
    plante_height = plante_img.get_height()
    plante_width = plante_img.get_width()

    # Ajuster le nombre de briques dans le tuyau supérieur si une plante est présente
    y_top = pipe_y - pipe_gap - brique_height
    min_y_top = -brique_height
    if has_plant:
        min_y_top += 2 * brique_height  # Retirer deux briques du tuyau supérieur
    while y_top > min_y_top:
        screen.blit(pygame.transform.flip(brique_img, False, True), (pipe_x, y_top))
        y_top -= brique_height

    # Dessiner les briques pour le tuyau inférieur (inchangé)
    y_bottom = pipe_y
    while y_bottom < SCREEN_HEIGHT:
        screen.blit(brique_img, (pipe_x, y_bottom))
        y_bottom += brique_height

    # Dessiner la plante si le tuyau a une plante
    if has_plant:
        screen.blit(plante_img, (pipe_x + (brique_width - plante_width) // 2, pipe_y - plante_height))

def main():
    pygame.init()
    pygame.display.set_caption(app_name)
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    start_screen(screen)

    # Initialisation des images
    bird_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/mario_volant.png').convert_alpha()
    bird_img = pygame.transform.scale(bird_img, (50, 50))
    brique_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/brique.png').convert_alpha()
    brique_img = pygame.transform.scale(brique_img, (50, 50))  # Redimensionner à 50x50
    plante_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/plante.png').convert_alpha()
    plante_img = pygame.transform.scale(plante_img, (50, 75))  # Ajustez la taille si nécessaire

    stars = create_stars(10)  # Création des étoiles

    video_played = False
    last_speed_increase_score = 0

    running = True
    while running:
        bird_y = BIRD_Y
        bird_velocity = 0
        # Ajouter 'has_plant' au premier tuyau
        pipes = [[300, random.randint(100, 400), False, random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX), random.choice([True, False])]]
        score = 0
        pipe_speed = PIPE_SPEED_INIT
        snowflakes = create_snowflakes(25)

        while running:
            # Gestion des événements
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        bird_velocity = FLAP_STRENGTH
                    elif event.key == pygame.K_DOWN:
                        bird_velocity = -FLAP_STRENGTH

            bird_velocity += GRAVITY
            bird_y += bird_velocity

            screen.fill((102, 190, 209))  # Couleur de fond
            draw_stars(screen, stars)  # Dessiner les étoiles
            update_and_draw_snowflakes(screen, snowflakes)  # Dessiner les flocons de neige
            screen.blit(bird_img, (BIRD_X, bird_y))  # Dessiner l'oiseau
            bird_rect = bird_img.get_rect(topleft=(BIRD_X, bird_y))  # Rectangle de l'oiseau pour les collisions

            # Gestion des tuyaux
            for pipe in pipes:
                pipe[0] += pipe_speed  # Mettre à jour la position x du tuyau
                pipe_x, pipe_y, pipe_passed, pipe_gap, has_plant = pipe

                # Dessiner le tuyau
                draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img, plante_img, has_plant)

                # Générer de nouveaux tuyaux
                if pipe[0] < -50:
                    new_gap = random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)  # Nouvel écart aléatoire
                    new_pipe_y = random.randint(100, 400)
                    has_plant = random.choice([True, False])  # Décider si le nouveau tuyau aura une plante
                    pipes.append([400, new_pipe_y, False, new_gap, has_plant])
                    pipes.pop(0)

                # Vérifier si le tuyau a été passé pour augmenter le score
                if pipe[0] + brique_img.get_width() < BIRD_X and not pipe_passed:
                    score += 1
                    pipe[2] = True  # Marquer le tuyau comme passé

            # Détection des collisions ajustée
            for pipe in pipes:
                pipe_x, pipe_y, pipe_passed, pipe_gap, has_plant = pipe
                pipe_width = brique_img.get_width()
                brique_height = brique_img.get_height()

                # Calculer la hauteur du tuyau supérieur
                upper_pipe_height = pipe_y - pipe_gap
                if has_plant:
                    upper_pipe_height -= 2 * brique_height  # Ajuster pour les deux briques en moins

                # Rectangle du tuyau supérieur
                upper_pipe_rect = pygame.Rect(pipe_x, 0, pipe_width, upper_pipe_height)

                # Rectangle du tuyau inférieur (inchangé)
                lower_pipe_rect = pygame.Rect(pipe_x, pipe_y, pipe_width, SCREEN_HEIGHT - pipe_y)

                # Vérifier la collision avec les tuyaux
                if bird_rect.colliderect(upper_pipe_rect) or bird_rect.colliderect(lower_pipe_rect):
                    running = False
                    break

                # Vérifier la collision avec la plante si elle est présente
                if has_plant:
                    plante_rect = plante_img.get_rect()
                    plante_rect.x = pipe_x + (pipe_width - plante_img.get_width()) // 2
                    plante_rect.y = pipe_y - plante_img.get_height()
                    if bird_rect.colliderect(plante_rect):
                        running = False
                        break

            draw_text(screen, f"Score: {score}", 24, SCREEN_WIDTH - 100, 50)  # Afficher le score

            # Augmenter la vitesse des tuyaux à intervalles réguliers
            if score // SCORE_INTERVAL_FOR_SPEED_INCREASE > last_speed_increase_score:
                pipe_speed -= 1  # Augmenter la vitesse (négatif car les tuyaux vont vers la gauche)
                last_speed_increase_score = score // SCORE_INTERVAL_FOR_SPEED_INCREASE

            # Jouer la vidéo lorsque le score atteint 10
            if score >= 10 and not video_played:
                play_video(screen, '/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/test_video.avi')
                video_played = True  # Éviter de rejouer la vidéo

            # Vérifier si l'oiseau est sorti de l'écran
            if bird_y >= SCREEN_HEIGHT - bird_img.get_height() or bird_y <= 0:
                running = False

            pygame.display.update()
            pygame.time.Clock().tick(30)  # Limiter à 30 FPS

        if not game_over_screen(screen, score):
            break  # Quitter si l'utilisateur choisit de ne pas recommencer

        running = True  # Recommencer le jeu
        video_played = False  # Réinitialiser pour le prochain jeu

    pygame.quit()


if __name__ == "__main__":
    main()


SystemExit: 

: 